# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read csv from first activity
og_df = pd.read_csv('output_data/cities.csv') 
#remove extra index
df = og_df.drop(columns=['Unnamed: 0']) 
df

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,leningradskiy,69.38,178.42,-15.41,96,100,8.81,1580409727
1,rikitea,-23.12,-134.97,79.59,75,100,13.87,1580409727
2,nanortalik,60.14,-45.24,17.91,60,0,27.47,1580409727
3,fortuna,40.60,-124.16,55.00,100,75,4.70,1580409727
4,katsuura,35.13,140.30,50.00,61,75,18.34,1580409727
5,kaeo,-35.10,173.78,63.00,100,1,3.27,1580409727
6,ilebo,-4.32,20.58,72.73,94,88,3.06,1580409728
7,feijo,-8.16,-70.35,82.44,84,96,4.14,1580409728
8,busselton,-33.65,115.33,48.99,52,0,7.90,1580409728
9,pevek,69.70,170.31,-16.69,98,100,0.72,1580409728


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = df["Humidity (%)"].astype(float)
maxhumidity = humidity.max()
locations = df[["Latitude", "Longitude"]]

In [4]:

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#narrow down df to ideal conditions, no null values will appear b/c they were removed by earlier code
narrowed_city_df = df.loc[(df["Max Temperature (F)"] > 70) & (df["Max Temperature (F)"] < 80) & (df["Cloudiness (%)"] < 50)]

lat = []
long = []

long = narrowed_city_df['Longitude']
lat = narrowed_city_df['Latitude']

narrowed_city_df

,City,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
12,saint-philippe,-21.36,55.77,78.8,74,20,6.93,1580409728
14,souillac,-20.52,57.52,78.8,78,40,3.36,1580409729
35,port macquarie,-31.43,152.92,75.0,94,5,5.82,1580409731


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# geocoordinates
target_coordinates = f"{lat},{long}"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()
print(response)

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}


In [ ]:
#couldnt figure out why the request is invalid? I gave up here because it is already overdue and i have other hw to work on but if whoever 
#grades this knows whats wrong id love to know why my api call is invalid

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map